This cell takes a csv file as input. From the file it reads a Time string, latitude, and longitude.
It converts the time to a timestamp.
Then it constructs and API call with the data and sends it over curl. The api returns a JSON string of the weather.
Pandas then processes the JSON weather and adds it to a new dataframe, and stores the results as new csv file.

This process took 10 hours to run with our dataset. Also it cost $8 for all the api calls. For one year of data.

In [6]:
import pandas as pd
import datetime as datetime
import time
import pycurl, certifi, json
from pandas.io.json import json_normalize
from io import BytesIO

In [7]:
df = pd.read_csv('311cleaned.csv')

In [8]:
df.head()

,REQUEST_ID,CREATED_ON,REQUEST_TYPE,REQUEST_ORIGIN,STATUS,DEPARTMENT,NEIGHBORHOOD,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,FIRE_ZONE,X,Y,GEO_ACCURACY
0,205290,2017-12-31T23:17:00,Snow/Ice removal,Website,1,DPW - Street Maintenance,Marshall-Shadeland,1.0,27.0,4.200327e+10,1.0,27.0,1.0,1-14,-80.034743,40.472661,EXACT
1,205289,2017-12-31T22:01:00,Snow/Ice removal,Report2Gov iOS,1,DPW - Street Maintenance,Stanton Heights,9.0,10.0,4.200310e+10,2.0,10.0,5.0,3-7,-79.936661,40.480244,EXACT
2,205288,2017-12-31T21:55:00,Snow/Ice removal,Website,1,DPW - Street Maintenance,New Homestead,5.0,31.0,4.200331e+10,3.0,31.0,4.0,4-20,-79.927033,40.389386,EXACT
3,205287,2017-12-31T18:23:00,Missed Pick Up,Website,1,DPW - Refuse,Mount Washington,2.0,19.0,4.200319e+10,5.0,19.0,3.0,4-27,-80.014213,40.426213,APPROXIMATE
4,205286,2017-12-31T17:56:00,Snow/Ice removal,Report2Gov iOS,1,DPW - Street Maintenance,Stanton Heights,9.0,10.0,4.200310e+10,2.0,10.0,5.0,3-7,-79.937035,40.480705,EXACT


In [9]:
df.describe()

,REQUEST_ID,STATUS,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,X,Y
count,79852.000000,79852.000000,76069.000000,75965.000000,7.612800e+04,75966.000000,75965.000000,75922.000000,76784.000000,76784.000000
mean,164666.873704,1.176514,4.882370,16.369157,4.200323e+10,3.029619,16.369157,3.550815,-79.289558,40.095406
std,23625.934393,0.703213,2.571621,8.025851,1.887250e+05,1.461813,8.025851,1.607305,7.360487,3.722107
min,123268.000000,0.000000,1.000000,0.000000,4.200301e+10,0.000000,0.000000,1.000000,-80.088105,0.000000
25%,144463.750000,1.000000,3.000000,10.000000,4.200311e+10,2.000000,10.000000,2.000000,-80.007287,40.421424
50%,164863.500000,1.000000,5.000000,16.000000,4.200318e+10,3.000000,16.000000,4.000000,-79.977465,40.444844
75%,185093.250000,1.000000,7.000000,22.000000,4.200328e+10,5.000000,22.000000,5.000000,-79.934367,40.461995
max,205290.000000,3.000000,9.000000,32.000000,4.200398e+10,6.000000,32.000000,6.000000,0.000000,40.526962


In [10]:
# https://darksky.net/dev/docs#time-machine-request
dsKey = "9aff7e03ca4b02e487cc0deac0dcbcab/";
dsUrl = "https://api.darksky.net/forecast/";
exclude = "?exclude=minutely,hourly,flags";
# dsAPI =  dsUrl + dsKey + dsLat + ',' + dsLon + ',' + Time + exclude
#takes 10 hours to process 80000 calls

In [15]:
dfwx = pd.DataFrame()  #new dataframe to store results
for index, row in df.iterrows():
#     if index > 10:  break #limit api calls 10000/$1

    #convert time string to unix time stamp
    date = datetime.datetime.strptime(row['CREATED_ON'], '%Y-%m-%dT%H:%M:%S') 
    date -= datetime.timedelta(seconds=14400) #convert back from utc
    timestamp = int(date.timestamp())
    
    #skip rows with no time/location data
    if not (timestamp and row['X'] and row['Y']):
        continue
    else:
        
        #write api calls to file for troubleshooting errors with jsondiff 
        dsAPI =  dsUrl + dsKey + str(row['Y']) + ',' + str(row['X']) + ',' + str(timestamp) + exclude
        with open('apicalls.txt', 'a+') as f:
            f.write(str(index) + ':\t' + dsAPI + "\n")    
        
        buffer = BytesIO()
        c = pycurl.Curl()
        c.setopt(c.URL, dsAPI)
        c.setopt(c.VERBOSE, True)
        c.setopt(c.CAINFO, certifi.where())
        c.setopt(c.WRITEDATA, buffer)
        c.perform()
        c.close()     
        wx = json.loads(buffer.getvalue().decode('iso-8859-1'))
        buffer.close()
    
        #should have been able to use json_normalize direct from that path. didn't work. Did it below.
        #https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas
        if 'daily' in wx:
            wxd = wx["daily"]["data"] 
            #wxd = json_normalize(wxd, record_path=[['daily', 'data']], record_prefix="daily.")
            wxd = json_normalize(wxd) #flattens json into series
            wxd = wxd.add_prefix('daily.')
        else: 
            wxd = pd.Series()

        #check if currently field and add 
        #add precipType field because it's not there if no precip and ruins frame alignment
        if 'currently' in wx:
            wxc = wx['currently']
            if 'precipType' not in wxc:
                wxc['precipType'] = ''
            wxc = json_normalize(wxc)
            wxc = wxc.add_prefix('current.')
        else: 
            wxc = pd.Series()
        
        new_row = pd.concat([pd.Series(row['REQUEST_ID'], name='REQUEST_ID'), wxc, wxd], axis=1, ignore_index=False)
        #print (index, dsAPI, wxd, wxc, new_row)
        if index % 100 == 0: print(index)
        dfwx = pd.concat([dfwx, new_row], axis=0, sort=False, ignore_index=False)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
1

In [16]:
dfwx.describe()

,REQUEST_ID,current.apparentTemperature,current.cloudCover,current.dewPoint,current.humidity,current.precipIntensity,current.precipProbability,current.pressure,current.temperature,current.time,...,daily.time,daily.uvIndex,daily.uvIndexTime,daily.visibility,daily.windBearing,daily.windGust,daily.windGustTime,daily.windSpeed,0,1
count,79196.000000,76128.000000,76128.000000,76128.000000,76128.000000,76128.000000,76128.000000,76128.000000,76128.000000,7.612800e+04,...,7.612800e+04,76128.000000,7.612800e+04,76128.000000,76128.000000,76128.000000,7.612800e+04,76128.000000,0.0,0.0
mean,164675.144439,55.038794,0.553601,47.324024,0.763721,0.003565,0.098312,1017.025954,55.172205,1.498397e+09,...,1.498365e+09,5.256568,1.498409e+09,9.378755,215.086381,9.185279,1.498414e+09,1.507024,NaN,NaN
std,23622.343937,17.396958,0.400407,16.534053,0.147205,0.013943,0.222132,6.846617,16.307025,8.216613e+06,...,8.216940e+06,2.402328,8.216563e+06,0.871242,74.410305,4.469073,8.218056e+06,1.140489,NaN,NaN
min,123268.000000,-0.080000,0.000000,-3.570000,0.160000,0.000000,0.000000,991.170000,1.230000,1.483262e+09,...,1.483247e+09,1.000000,1.483290e+09,3.780000,0.000000,1.460000,1.483247e+09,0.020000,NaN,NaN
25%,144485.750000,43.367500,0.110000,35.480000,0.670000,0.000000,0.000000,1013.780000,44.060000,1.491909e+09,...,1.491883e+09,3.000000,1.491937e+09,9.080000,186.000000,5.690000,1.491926e+09,0.700000,NaN,NaN
50%,164868.500000,58.130000,0.650000,51.120000,0.800000,0.000000,0.000000,1017.170000,58.130000,1.498470e+09,...,1.498450e+09,5.000000,1.498489e+09,9.760000,224.000000,8.720000,1.498511e+09,1.290000,NaN,NaN
75%,185099.250000,67.540000,1.000000,60.830000,0.890000,0.000300,0.030000,1020.610000,67.100000,1.504735e+09,...,1.504670e+09,7.000000,1.504714e+09,10.000000,266.000000,11.560000,1.504750e+09,2.080000,NaN,NaN
max,205290.000000,96.380000,1.000000,74.060000,0.980000,0.268400,1.000000,1041.070000,90.490000,1.514766e+09,...,1.514696e+09,10.000000,1.514732e+09,10.000000,359.000000,26.410000,1.514696e+09,6.150000,NaN,NaN


In [17]:
dfwx.sample(5)

,REQUEST_ID,current.apparentTemperature,current.cloudCover,current.dewPoint,current.humidity,current.icon,current.precipIntensity,current.precipProbability,current.precipType,current.pressure,...,daily.time,daily.uvIndex,daily.uvIndexTime,daily.visibility,daily.windBearing,daily.windGust,daily.windGustTime,daily.windSpeed,0,1
0,181729,74.54,0.75,55.18,0.51,partly-cloudy-day,0.0000,0.00,,1014.77,...,1.503461e+09,6.0,1.503504e+09,9.72,287.0,6.99,1.503518e+09,0.85,NaN,NaN
0,169716,76.35,0.59,69.73,0.83,partly-cloudy-day,0.0000,0.00,,1017.55,...,1.499918e+09,5.0,1.499965e+09,9.28,222.0,12.01,1.499972e+09,1.33,NaN,NaN
0,139605,23.25,0.92,16.95,0.61,partly-cloudy-day,0.0000,0.00,,1024.77,...,1.489637e+09,4.0,1.489684e+09,9.67,280.0,13.84,1.489691e+09,3.37,NaN,NaN
0,141142,53.01,1.00,50.85,0.92,rain,0.0501,0.84,rain,1014.47,...,1.490587e+09,5.0,1.490630e+09,9.71,233.0,9.86,1.490641e+09,1.53,NaN,NaN
0,130630,60.29,0.91,53.00,0.77,rain,0.0045,0.47,rain,1001.34,...,1.486444e+09,2.0,1.486483e+09,9.12,212.0,13.97,1.486516e+09,3.19,NaN,NaN


In [18]:
with open('311weather.csv', 'w') as fh:
    dfwx.to_csv(fh, index=False)